# Convert Parquet to Delta Lake

In [7]:
import pyspark
from delta import *

In [9]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [11]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/12/24 15:16:48 WARN Utils: Your hostname, Prabhakaras-Mac-Book-Pro-2022.local resolves to a loopback address: 127.0.0.1; using 192.168.0.137 instead (on interface en0)
24/12/24 15:16:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/prabhakarapelluru/.ivy2/cache
The jars for the packages stored in: /Users/prabhakarapelluru/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9046566e-f2f7-44cd-b449-ec471d94ab0d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 67ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	-----------------------------------

:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


24/12/24 15:16:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 15:16:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/24 15:16:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/12/24 15:16:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/12/24 15:16:49 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/12/24 15:16:49 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/12/24 15:16:49 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [13]:
columns = ["language", "num_speakers"]
data = [("English", "1.5"), ("Mandarin", "1.1"), ("Hindi", "0.6")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

## Unpartitioned Parquet to Delta Lake

In [20]:
df.write.format("parquet").save("data/lake1")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/prabhakarapelluru/prabhakara/data_science/code/04-datascience-spark-python-2024/00-prabha-delta-lake-examples/03-parquet-2-deltalake/data/lake1 already exists. Set mode as "overwrite" to overwrite the existing path.

In [22]:
!tree data/lake1

data/lake1
├── _SUCCESS
├── part-00000-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet
├── part-00002-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet
├── part-00005-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet
└── part-00007-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet

1 directory, 5 files


In [24]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`data/lake1`")

24/12/24 15:18:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [28]:
!tree data/lake1

data/lake1
├── _SUCCESS
├── _delta_log
│   ├── 00000000000000000000.checkpoint.parquet
│   ├── 00000000000000000000.json
│   ├── _commits
│   └── _last_checkpoint
├── part-00000-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet
├── part-00002-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet
├── part-00005-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet
└── part-00007-1c8b5145-64bc-445f-86bd-fdca02cbe8d1-c000.snappy.parquet

3 directories, 8 files


## Partitioned Parquet to Delta Lake

In [30]:
df.write.partitionBy("language").format("parquet").save("data/lake2")

In [32]:
!tree data/lake2

data/lake2
├── _SUCCESS
├── language=English
│   └── part-00002-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet
├── language=Hindi
│   └── part-00007-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet
└── language=Mandarin
    └── part-00005-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet

4 directories, 4 files


In [34]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`data/lake2`")

AnalysisException: [DELTA_UNEXPECTED_NUM_PARTITION_COLUMNS_FROM_FILE_NAME] Expecting 0 partition column(s): [], but found 1 partition column(s): [`language`] from parsing the file name: file:/Users/prabhakarapelluru/prabhakara/data_science/code/04-datascience-spark-python-2024/00-prabha-delta-lake-examples/03-parquet-2-deltalake/data/lake2/language=English/part-00002-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet

In [36]:
!tree data/lake2

data/lake2
├── _SUCCESS
├── _delta_log
│   └── _commits
├── language=English
│   └── part-00002-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet
├── language=Hindi
│   └── part-00007-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet
└── language=Mandarin
    └── part-00005-6eb4d5fd-bb5e-49e0-b112-035555e1757b.c000.snappy.parquet

6 directories, 4 files


In [ ]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`data/lake2`", "language STRING")

In [50]:
!tree tmp/lake2

tmp/lake2
├── _SUCCESS
├── _delta_log
│   ├── 00000000000000000000.checkpoint.parquet
│   ├── 00000000000000000000.json
│   └── _last_checkpoint
├── language=English
│   └── part-00003-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet
├── language=Hindi
│   └── part-00009-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet
└── language=Mandarin
    └── part-00006-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet

4 directories, 7 files


## Convert Parquet Lake with Multiple Appends

In [ ]:
df.write.format("parquet").save("data/lake3")

In [51]:
columns = ["language", "num_speakers"]
data = [("Spanish", "0.5"), ("French", "0.3"), ("Arabic", "0.3")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [52]:
df.write.mode("append").format("parquet").save("data/lake3")

In [53]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`data/lake3`")

In [56]:
deltaTable.version(0).show()

AttributeError: 'DeltaTable' object has no attribute 'version'

## Convert Parquet Lake no statistics

In [16]:
df.write.format("parquet").saveAsTable("lake4")

In [20]:
spark.sql("CONVERT TO DELTA lake4 NO STATISTICS")

ParseException: 
Syntax error at or near 'CONVERT'(line 1, pos 0)

== SQL ==
CONVERT TO DELTA lake4 NO STATISTICS
^^^


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52063)
Traceback (most recent call last):
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in ha

## Cleanup